## ETL Pipeline for Pre-Processing the Files

#### PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

file_path_list = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    for name in files:
        file_path_list.append(os.path.join(root, name))
# join the file path and roots with the subdirectories using glob
#     file_path_list.append(glob.glob(os.path.join(root, file)))
#     print(file_path_list)

/home/workspace


In [3]:
print(len(file_path_list))

31


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_data_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_data_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_data_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6832


## The event_data_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_data_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

Replication factor of one means that there is only one copy of each row in the Cassandra cluster. A replication factor of two means there are two copies of each row, where each copy is on a different node. As a general rule, the replication factor should not exceed the number of Cassandra nodes in the cluster.

SimpleStrategy: Use only for a single datacenter and one rack. If we were to ever use more than one datacenter, use the NetworkTopologyStrategy.

A datacenter is a group of related nodes that are configured together within a cluster for replication and workload segregation purposes. Not necessarily a separate location or physical data center. Datacenter names are case sensitive and cannot be changed.

In [7]:
# Create a Keyspace
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        { 'class': 'SimpleStrategy', 'replication_factor': 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

In Apache Cassandra, we model the database tables on the queries we want to run.

## Queries to ask the following 3 questions of the data

### 1. Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [13]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS user_session_records"
query = query + "(artist text, song_title text, song_length float, session_id int, item_in_session int, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_records (artist, song_title, song_length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        """
        Assign which column element should be assigned for each column in the INSERT statement.
        For e.g., to INSERT artist_name and user first_name, you would change the code below
        to `line[0], line[1]`
        """
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))



In [16]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, song_length FROM user_session_records WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [17]:
query = "CREATE TABLE IF NOT EXISTS music_session_records"
query = query + "(artist text, song_title text, user_id int, session_id int, item_in_session int, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session, first_name, last_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_session_records (artist, song_title, user_id, session_id, item_in_session, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[10]), int(line[8]), int(line[3]), line[1], line[4]))


In [19]:
query = "SELECT artist, song_title, first_name, last_name FROM music_session_records WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Return every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In [20]:
query = "CREATE TABLE IF NOT EXISTS song_listening_records"
query = query + "(song_title text, first_name text, last_name text, PRIMARY KEY (song_title))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [21]:
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listening_records (song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))



In [22]:
query = "SELECT first_name, last_name FROM song_listening_records WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

Tegan Levine


### Drop the tables before closing out the sessions

In [23]:
# Drop the table before closing out the sessions
query1 = "DROP TABLE user_session_records"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

query2 = "DROP TABLE music_session_records"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

query3 = "DROP TABLE song_listening_records"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()